# Helper function to convert number to friendly string

In [2]:
def human_readable(num):
    num = float(num)  # make sure it’s numeric
    if num >= 1_000_000_000_000:   # Trillions
        return f"{num/1_000_000_000_000:.1f} T"
    elif num >= 1_000_000_000:     # Billions
        return f"{num/1_000_000_000:.1f} B"
    elif num >= 1_000_000:         # Millions
        return f"{num/1_000_000:.1f} M"
    elif num >= 1_000:             # Thousands
        return f"{num/1_000:.0f} K"
    else:
        return str(num)


In [ ]:
from pathlib import Path
import json

def write_ndjson_local(country_gw: int, year: int, page: int, rows: list, base_dir="data"):
    """
    Write a page of UCDP events to an NDJSON file under data/raw/.

    Structure:
      data/
        raw/
          country=090/
            year=1989/
              page=00001.ndjson
    """
    base = Path(base_dir) / "raw" / f"country={country_gw:03d}" / f"year={year}"
    base.mkdir(parents=True, exist_ok=True)

    path = base / f"page={page:05d}.ndjson"
    with path.open("w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r) + "\n")

    return str(path)

In [ ]:
%%sql


# Getting GDP information from WorldBank

In [9]:
import requests
import pandas as pd
import country_converter as coco

BASE = "https://api.worldbank.org/v2/country/{country}/indicator/{indicator}"


COUNTRIES = ["United States", "Israel", "Moldova"]
INDICATORS = {
    "NY.GDP.MKTP.CD": "gdp_usd",
    "SP.POP.TOTL": "population",
    "MS.MIL.XPND.GD.ZS": "mil_exp_pct_gdp",
    "MS.MIL.XPND.CD": "mil_exp_usd",
}

START, END = 2012, 2023

def get_wb_code(country_name):
    try:
        return coco.convert(country_name, to='ISO3')
    except:
        return None  # Handle countries that don't convert


def fetch_indicator(country, start, end):
    merged = None
    for code, colname in INDICATORS.items():

        wb_code = get_wb_code(country)
        print(wb_code)
        url = BASE.format(country=wb_code, indicator=code)
        params = {"format": "json", "date": f"{start}:{end}", "per_page": 20000}
        r = requests.get(url, params=params, timeout=30)
        r.raise_for_status()
        payload = r.json()
        rows = payload[1] if isinstance(payload, list) and len(payload) > 1 else []

        df = pd.DataFrame([
            {
                "country": row["country"]["value"],
                "iso3": row["countryiso3code"],
                "year": int(row["date"]),
                colname: row["value"],
            }
            for row in rows if row.get("value") is not None
        ])

        # merge indicator onto main DataFrame
        if merged is None:
            merged = df
        else:
            merged = pd.merge(merged, df, on=["country", "iso3", "year"], how="outer")

    merged['gdp_string'] = merged['gdp_usd'].apply(human_readable)
    merged['population_string'] = merged['population'].apply(human_readable)
    return merged.sort_values("year").reset_index(drop=True)


# ---------------- RUN ----------------
frames = [fetch_indicator(c, START, END) for c in COUNTRIES]
df = pd.concat(frames).sort_values(["year"]).reset_index(drop=True)

df.head(50)
# df.to_csv("gdp_countries.csv", index=False)


USA
USA
USA
USA
ISR
ISR
ISR
ISR
MDA
MDA
MDA
MDA


,country,iso3,year,gdp_usd,population,mil_exp_pct_gdp,mil_exp_usd,gdp_string,population_string
0,United States,USA,2012,1.625397e+13,314339099,4.461710,7.252050e+11,16.3 T,314.3 M
1,Israel,ISR,2012,2.635892e+11,7910500,5.641973,1.452442e+10,263.6 B,7.9 M
2,Moldova,MDA,2012,8.709139e+09,2860324,0.274080,2.387007e+07,8.7 B,2.9 M
3,Israel,ISR,2013,2.984781e+11,8059500,5.458969,1.624783e+10,298.5 B,8.1 M
4,Moldova,MDA,2013,9.496718e+09,2859558,0.281262,2.671052e+07,9.5 B,2.9 M
5,United States,USA,2013,1.688068e+13,316726282,4.023706,6.792290e+11,16.9 T,316.7 M
6,United States,USA,2014,1.760814e+13,319257560,3.678918,6.477890e+11,17.6 T,319.3 M
7,Israel,ISR,2014,3.150171e+11,8215700,5.647197,1.773738e+10,315.0 B,8.2 M
8,Moldova,MDA,2014,9.402090e+09,2857815,0.293185,2.756556e+07,9.4 B,2.9 M
9,Moldova,MDA,2015,7.797667e+09,2835978,0.295625,2.305181e+07,7.8 B,2.8 M


In [ ]:
# Must have columns include
from pathlib import Path
import time, json
'''
id
country_id
year
country
ged_events (This will basically count the number of incidents in a year)
ged_events_fatal (This will basically the events that ended up being fatal)
ged_deaths_best (This will show the best estimate on how many people were killed
ged_deaths_civilians (This will be the sum of all of the civilian deaths)
ged_deaths_low (This will be a sum of all of the low death estimates)
ged_deaths_high (This will be a sum of all of the high death estimates)
ged_dyads (This will be a count of all the dyad_new_id that we have)
ged_state_events (This will show the events that have 1)
ged_nonstate_events (This will show the events that have 2)
ged_onesided_events (This will show the events that have 3)
'''


from pathlib import Path
import time, json

def save_raw(df):
    """Save the raw API rows to JSONL (raw layer)."""
    raw_base = Path("data/raw/gedevents")
    raw_base.mkdir(parents=True, exist_ok=True)

    ts = int(time.time())
    raw_path = raw_base / f"all_countries_part-{ts}.jsonl"

    with open(raw_path, "w", encoding="utf-8") as f:
        for _, row in df.iterrows():
            f.write(json.dumps(row.dropna().to_dict(), ensure_ascii=False, default=str) + "\n")

    print(f"✅ Wrote RAW JSONL to {raw_path}")


def save_processed(agg):
    """Save the aggregated country-year data to partitioned Parquet (processed layer)."""
    processed_base = Path("data/processed/gedevents")
    processed_base.mkdir(parents=True, exist_ok=True)

    ts = int(time.time())
    rows_written = 0

    for (yr, cid), part in agg.groupby(["year", "country_id"], dropna=False):
        out_dir = processed_base / f"year={int(yr)}" / f"country_id={int(cid)}"
        out_dir.mkdir(parents=True, exist_ok=True)
        out_path = out_dir / f"part-{ts}.parquet"
        part.to_parquet(out_path, index=False, engine='fastparquet')
        rows_written += len(part)

    print(f"✅ Wrote {rows_written} rows to Parquet under {processed_base}/")



# Getting information from UCDP


In [ ]:
import requests, time, json
import pandas as pd

BASE_URL = "https://ucdpapi.pcr.uu.se/api/gedevents/25.1"

GW_CODES = [645, 700, 775, 540, 666]

all_events = []

for country in GW_CODES:
    print(f"\n=== Starting retrieval for country code: {country} ===")

    url = BASE_URL

    params = {
        "Country": country,
        "StartDate": "1989-01-01",
        "EndDate": "1991-12-31",
        "pagesize": 1000
    }


    while True:
        r = requests.get(url, params=params if url == BASE_URL else None, timeout=30)
        r.raise_for_status()
        payload = r.json()

        events = payload.get("Result", [])
        all_events.extend(events)

        next_url = payload.get("NextPageUrl")
        if not next_url:
            break
        url, params = next_url, None  # NextPageUrl already includes query params
        time.sleep(0.2)               # gentle pacing

    print("\n--- Retrieval Complete ---")
    print(f"Total events retrieved: {len(all_events)}")

# Build DataFrame
df = pd.DataFrame(all_events)

# Parse dates & cast numerics safely
df["date_start"] = pd.to_datetime(df.get("date_start"), errors="coerce")
for col in ["best", "low", "high", "deaths_civilians"]:
    df[col] = pd.to_numeric(df.get(col), errors="coerce").fillna(0).astype(int)

# Aggregate to Country–Year
agg = (
    df.groupby(["country_id", "country", "year"], dropna=False)
      .agg(
          ged_events          = ("id", "count"),
          ged_events_fatal    = ("best", lambda x: (x > 0).sum()),
          ged_deaths_best     = ("best", "sum"),
          ged_deaths_low      = ("low", "sum"),
          ged_deaths_high     = ("high", "sum"),
          ged_deaths_civilians= ("deaths_civilians", "sum"),
          ged_dyads           = ("dyad_new_id", "nunique"),
          ged_state_events    = ("type_of_violence", lambda x: (x == 1).sum()),
          ged_nonstate_events = ("type_of_violence", lambda x: (x == 2).sum()),
          ged_onesided_events = ("type_of_violence", lambda x: (x == 3).sum()),
      )
      .reset_index()
      .sort_values(["country_id", "year"])
)

save_raw(df)
save_processed(agg)

# # Chronological preview of raw events
# df_sorted = df.sort_values(["year", "date_start", "id"])
# print("\nFirst 5 events chronologically:")
# print(df_sorted[["id","year","country","date_start","best","deaths_civilians"]].head(5))
#
# print("\nLast 5 events chronologically:")
# print(df_sorted[["id","year","country","date_start","best","deaths_civilians"]].tail(5))
